In [ ]:
import pandas as pd
file_path = '/home/yuhaowang/project/FMBC/downstream/finetune/dataset_csv/others'
csv_path = 'GTEX_BreastGENE.csv'
data = pd.read_csv(csv_path)
print(len(data.columns))
data.head()

5484


,silde_id,TSPAN6,TNMD,DPM1,SCYL3,C1orf112,FGR,CFH,FUCA2,GCLC,...,TOX2,SRSF6,GDAP1L1,GTSF1L,ARFGEF2,ZNFX1,ZNF831,EDN3,CSE1L,TMEM189-UBE2V1
0,GTEX-ZA64-1526,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.822439,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,GTEX-X4EP-2926,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.758760,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,GTEX-S7SE-0826,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,GTEX-XMD1-0826,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,GTEX-11P81-192,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.814639,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [11]:
import pandas as pd
import os
file_path = '/home/yuhaowang/project/FMBC/downstream/finetune/dataset_csv/others'
for file in os.listdir(file_path):
    print(file)
    data = pd.read_csv(os.path.join(file_path, file))
    print(len(data.columns))

Multi-omic_RNASEQ.csv
57909
TCGA-BRCA_GENECOUNTS.csv
59428
TIGER-TIL-SCORES-WSITILS.csv
2
TCGA-BRCA_FPKMS_LOG2.csv
59428
BreakHis_MALIGNANT_4SUBTYPE.csv
2
CPTAC_GENE.csv
8344
TCGA-BRCA_TPMS_LOG2.csv
59428
GTEX_BreastGENE.csv
5484
BreakHis_BENIGN_4SUBTYPE.csv
2


/tmp/ipykernel_1049146/3998511732.py:6: DtypeWarning: Columns (0,1) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv(os.path.join(file_path, file))


In [1]:
import pandas as pd
import numpy as np

# 读取原始CSV文件
input_file = "/home/yuhaowang/project/FMBC/downstream/finetune/dataset_csv/survival/TCGA-BRCA-SURVIVAL.csv"  # 请替换为您的输入文件路径
output_file = "/home/yuhaowang/project/FMBC/downstream/finetune/dataset_csv/survival/TCGA-BRCA-SUR-NEW.csv"  # 输出文件路径
df = pd.read_csv(input_file)

# 统一列名
column_mapping = {
    "slide_id": "case_id",
    "slide_id1": "slide_id",
    "survival_months": "OS_days",
    "censorship": "OS_event",
    "disc_label": "OS"
}
df.rename(columns=column_mapping, inplace=True)

# 将生存时间（OS_days）从月转换为天
df["OS_days"] = (df["OS_days"] * 30.4375).astype(int)  # 近似换算为天

# 计算生存四分位数（Quartile）
quantiles = df["OS_days"].quantile([0.25, 0.5, 0.75]).values

# 根据OS_days分配Quartile

def assign_quartile(os_days):
    if os_days <= quantiles[0]:
        return 0
    elif os_days <= quantiles[1]:
        return 1
    elif os_days <= quantiles[2]:
        return 2
    else:
        return 3

df["OS_quartile"] = df["OS_days"].apply(assign_quartile)

# 生成最终的OS类别

df["OS"] = df["OS_quartile"] + df["OS_event"] * 4

df.drop(columns=["OS_quartile"], inplace=True)

# 添加交叉验证折叠列（这里随机分配，5折交叉验证）
num_folds = 5
df[[f"fold_{i}" for i in range(num_folds)]] = "train"
df.loc[df.sample(frac=1 / num_folds).index, f"fold_{num_folds - 1}"] = "test"

# 保存转换后的CSV文件
df.to_csv(output_file, index=False)
print(f"转换完成，已保存为 {output_file}")


转换完成，已保存为 /home/yuhaowang/project/FMBC/downstream/finetune/dataset_csv/survival/TCGA-BRCA-SUR-NEW.csv


In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import pandas as pd
import numpy as np
import h5py
import matplotlib.pyplot as plt
from torch.utils.data import Dataset, DataLoader
from lifelines.utils import concordance_index
from lifelines import KaplanMeierFitter

# 1️⃣ 自定义Dataset类
class SurvivalDataset(Dataset):
    def __init__(self, csv_file, h5_folder):
        self.data = pd.read_csv(csv_file)
        self.h5_folder = h5_folder
        self.data['OS_days'] = (self.data['survival_months'] * 30.4375).astype(int)  # 月转换为天
        
    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, idx):
        row = self.data.iloc[idx]
        slide_id = row['slide_id1']
        h5_path = f"{self.h5_folder}/{slide_id}.h5"
        
        with h5py.File(h5_path, 'r') as h5_file:
            features = np.array(h5_file['features'])  # 读取h5文件中的特征
        
        features = torch.tensor(features, dtype=torch.float32)
        OS_days = torch.tensor(row['OS_days'], dtype=torch.float32)
        OS_event = torch.tensor(row['censorship'], dtype=torch.float32)  # 1=死亡, 0=删失数据
        return features, OS_days, OS_event

# 2️⃣ 定义一个简单的MLP模型
class SurvivalModel(nn.Module):
    def __init__(self, input_dim):
        super(SurvivalModel, self).__init__()
        self.fc = nn.Sequential(
            nn.Linear(input_dim, 128),
            nn.ReLU(),
            nn.Linear(128, 1)
        )
    
    def forward(self, x):
        return self.fc(x).squeeze()

# 3️⃣ 负对数似然损失（Cox回归损失）
def neg_log_partial_likelihood(preds, times, events):
    risk_order = torch.argsort(times, descending=True)
    preds, times, events = preds[risk_order], times[risk_order], events[risk_order]
    
    log_risk = torch.log(torch.cumsum(torch.exp(preds), dim=0))
    uncensored_likelihood = preds - log_risk
    loss = -torch.sum(uncensored_likelihood * events) / events.sum()
    return loss

# 4️⃣ 训练与评估函数
def train_survival_model(csv_file, h5_folder, input_dim, epochs=10, batch_size=4, lr=1e-3):
    dataset = SurvivalDataset(csv_file, h5_folder)
    dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)
    
    model = SurvivalModel(input_dim)
    optimizer = optim.Adam(model.parameters(), lr=lr)
    
    for epoch in range(epochs):
        model.train()
        total_loss = 0
        times_all, preds_all, events_all = [], [], []
        
        for features, times, events in dataloader:
            optimizer.zero_grad()
            preds = model(features)
            loss = neg_log_partial_likelihood(preds, times, events)
            loss.backward()
            optimizer.step()
            total_loss += loss.item()
            
            times_all.extend(times.tolist())
            preds_all.extend(preds.tolist())
            events_all.extend(events.tolist())
        
        # 计算C-Index
        c_index = concordance_index(times_all, preds_all, events_all)
        print(f"Epoch {epoch+1}/{epochs}, Loss: {total_loss:.4f}, C-Index: {c_index:.4f}")
    
    return model

# 5️⃣ 统计分析与KM生存曲线
def plot_kaplan_meier(csv_file):
    data = pd.read_csv(csv_file)
    data['OS_days'] = (data['survival_months'] * 30.4375).astype(int)  # 转换为天
    kmf = KaplanMeierFitter()
    
    plt.figure(figsize=(8,6))
    for label in sorted(data['disc_label'].unique()):
        group = data[data['disc_label'] == label]
        kmf.fit(group['OS_days'], event_observed=group['censorship'], label=f"Group {label}")
        kmf.plot_survival_function()
    
    plt.title("Kaplan-Meier Survival Curves")
    plt.xlabel("Days")
    plt.ylabel("Survival Probability")
    plt.legend()
    plt.show()

# 6️⃣ 运行示例
if __name__ == "__main__":
    csv_file = "data/survival_data.csv"  # 替换成您的CSV文件路径
    h5_folder = "data/h5_features"  # 替换成您的H5特征存储文件夹
    input_dim = 512  # 假设每个H5文件包含512维的特征向量
    
    trained_model = train_survival_model(csv_file, h5_folder, input_dim)
    plot_kaplan_meier(csv_file)


In [ ]:
import pandas as pd
data = pd.read_csv(csv_file)

In [3]:
data = pd.read_csv('/home/yuhaowang/project/FMBC/downstream/finetune/dataset_csv/survival/TCGA-BRCA-SURVIVAL.csv')
#drop column slide_id1	
data = data.drop('slide_id1', axis=1)

In [6]:
data['OS_days'] = (data['survival_months'] * 30.4375).astype(int)
data.to_csv('TCGA-BRCA-SURVIVAL.csv', index=False)